# Management and Analysis of Physics Dataset - mod.B

## Final project: Streaming processing of cosmic rays using Drift Tubes detectors

The goal of this project is to reproduce a real-time processing of real data collected in a particle physics detector and publish the results in a dashboard for live monitoring.

### Students:
* Conforto Filippo (2021856)
* Domenichetti Lorenzo (missing)
* Faorlin Tommaso (2021857)

## Consumer notebook

In [9]:
import json
import pylab as pl
import numpy as np

from IPython import display

from kafka import KafkaConsumer

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
from kafka.admin import KafkaAdminClient, NewTopic
KAFKA_BOOTSTRAP_SERVERS='10.67.22.100:9092'
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    )

consumer = KafkaConsumer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,consumer_timeout_ms=100000)

consumer.subscribe('topic_results')

consumer.poll(timeout_ms=0,         #<<--- do not enable dead-times before one poll to the next
              max_records=None,     #<<--- do not limit the number of records to consume at once 
              update_offsets=True   #<<--- update the reading offsets on this topic
             )

{}

In [11]:
def rt_plot(js,num, fig,axes):
    
    for txt in fig.texts:
        txt.set_visible(False)
    
    plt.figtext(1.1, 0.5, f'Batch N° {num}:\n\n Hits: {js["hits"]} \n\n' + f'Channel 1: {js["hits_per_chamber"][0]} hits\n\n ' + f'Channel 2: {js["hits_per_chamber"][1]} hits\n\n' + f'Channel 3: {js["hits_per_chamber"][2]} hits\n\n' + f'Channel 4: {js["hits_per_chamber"][3]} hits' 
                , ha='center', va='center', fontsize = 20,
               bbox=dict(facecolor='none', edgecolor='black', boxstyle='round'))
    
    plt.figtext(0.5, 1.025, f'Total active channels', ha='center', va='center', fontsize = 22)
                
    for i, ax in enumerate(axes[0]):
        ax.clear()
        ax.set_title(f'Chamber {i}', fontsize = 15)
        ax.set_xlabel("Channel N°", fontsize = 14)
        if i == 0:
            ax.set_ylabel("Counts", fontsize = 14)
        bin_centers = js["hist_1"][str(i)]["bins"][:-1] + np.diff(js["hist_1"][str(i)]["bins"])/2
        ax.hist(bin_centers, weights=js["hist_1"][str(i)]["counts"], bins=js["hist_1"][str(i)]["bins"], alpha=0.6)
        ax.tick_params(labelsize=13)
                
    for i, ax in enumerate(axes[1]):
        ax.clear()
        ax.set_title(f'Chamber {i}', fontsize = 15)
        ax.set_xlabel("Orbit N°", fontsize = 14)
        if i == 0:
            ax.set_ylabel("Counts", fontsize = 14)
        bin_centers = js["hist_2"][str(i)]["bins"][:-1] + np.diff(js["hist_2"][str(i)]["bins"])/2
        ax.hist(bin_centers, weights=js["hist_2"][str(i)]["counts"], bins=js["hist_2"][str(i)]["bins"], alpha=0.6)
        ax.tick_params(labelsize=13)
        ax.ticklabel_format(axis="x", style="sci", scilimits=(1,1)) 
                
    plt.figtext(0.5, 0.5, 'Total number of active channels per orbit', ha='center', va='center', fontsize = 22)
    plt.tight_layout(h_pad = 7, w_pad = 1)
    display.clear_output(wait=True)
    display.display(pl.gcf())

In [ ]:
fig, axes = plt.subplots(2,4, figsize = (15,10), sharey = 'row')
for num,message in enumerate(consumer):
    rt_plot(json.loads(message.value),num,fig,axes)